# Sendig emails with Python

Sending dozens or hundreds of emails with the same structure but somehow personalized can be really time 
consuming if you do it manually. In one project, we needed someone to do exactly this. 
To be more specific, the task was to open log file, extract some information and contact 
respective person by email using extracted data.

I tried this task once on my own – it took me more than a minute to complete all steps for one email. 
Imagine you have to send 100 such emails per day… Well, if you do some homework, you can get rid of this 
by using some sort of script. This article is about using Python and Outlook to do such activities. 
Let’s begin!


## Few Notes
Note 1: If you don’t already have Python installed in your computer, I suggest installing latest Anaconda distribution – link here https://docs.anaconda.com/anaconda/install/

Note 2: This tutorial is about using Python and Outlook. This means that all emails will be sent from your email address, which you have specified in Outlook. It may be that some exchange server setting might prevent some actions. Shall this happen, check with your IT administrators. 

Note 3: Sending email using Python is actually pretty simple. Just use few lines from cell below. However, this wouldn’t be so useful without having some additional code that is doing the heavy lifting for you. Therefore, let me go through steps I did in my case and I invite you to consider what would be right approach in your situation.

In [ ]:
import win32com.client as win32
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = receiver
mail.Subject = subject
mail.Body = body
mail.Send()

# Import pandas

In [4]:
import pandas as pd

# read csv files - make sure they are located in the same folder as this notebook. Otherwise specify
# exact path to the location of the files

df = pd.read_csv('log02.csv') 
df_em = pd.read_csv('emails.csv')

In [6]:
# let's see how the data looks like

df.head(2)
#df_em

,User:,Date:,IP:,Hostname:,Description:
0,jjksdf,27.10.2020,10.35.16.124,HSTDFEWPEW,Described what doing
1,vcvsd,27.10.2020,10.35.16.125,DEFDSDSD0101010,Described what doing not


# Design of solution
Always consider your situation with all specifics before copy/pasting from my solution. Let me outline what was my situation and how I approached it.
We needed to check log files in order to extract who connected to certain infrastructure and then contact respective administrator to and ask some for some details. Logs were provided in csv file and contained an identifier of admin, date/time information of access to server and many other details. For demonstration I’m using just a sample file with made up entries. 

I’m using separate file where each admin identifier is linked to email address but in case you have excel exports, this may be stored in same file but different tab. Shall this solution be used in bigger scale, then I would use database solution. Nevertheless, this was just a small exercise run by me so I think using csv or excel files is just fine.

Each email can be broken down into few fields you should provide. Here, I will consider subject, body of email and of course, the receiver. Each is handled by separate function that will be triggered.

In [2]:
def subj(receiver,date,hostname):
    # split receiver email address 
    use_receiver = receiver.split('@')[0].split('.')

    #creacte subject message - use first 2 elements of use_receiver (name and surname)
    subject = f'{use_receiver[0]} {use_receiver[1]} worked on {hostname} at {date} - confirm action'
    
    return subject

In [3]:
def body(receiver,date,hostname):
    # split receiver email address 
    use_receiver = receiver.split('@')[0].split('.')
    
    #creacte subject message - use first 2 elements of use_receiver (name and surname)
    body = f"""Dear {use_receiver[0]} {use_receiver[1]}, 
    
you have worked on: ' {hostname} ' at {date}. Please provide reference...........
    
Thank you.

Regards,
XYZ
    """
    
    return body

In [1]:
def send_email(receiver,subject,body):
    """
    Sending email via outlook app
    
    receiver - email address of the receiver and must be string. For more recipients, separate them by ; but 
    still must be part of the same string
    subject - subject of the email, string
    body - body of the email, string
    """
    import win32com.client as win32 
    
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = receiver
    mail.Subject = subject
    mail.Body = body
    mail.Send()

# Let’s start sending emails
With data loaded, functions defined, let’s actually send some emails. We can apply functions for all entries in the log file with simple “for” loop.

At the beginning, we need to get user’s email address – indexing herein may be little confusing at first look, so be careful. Next is obtaining date and hostname info.

Once done, we can call the functions which will do the heavy lifting. Please note that last command of the loop is time.sleep(1). This will make sure that once email is sent, there is a short break before sending another email. The reason for this is to avoid overloading exchange server with too many requests.


In [191]:
# loop to run actual email sending

import time
a = []

for i in range(0,len(df)):
    
    # get user's email address
    a.append(df['User:'][i])
    b = a[i]
    receiver = str(df_em[df_em['wiw']==b]['email']).split()[1]
   
    
    #obtain date and hostname data
    date = df['Date:'][i]
    hostname = df['Hostname:'][i]
    
    #call subj function to create subject for email
    subject = subj(receiver,date,hostname)
    
    # call body func to create body of the email
    e_body = body(receiver,date,hostname)
    
    # send actual email and wait before next iteration
    send_email(receiver,subject,e_body)
    time.sleep(1)
    